# Chapter 9: EM Algorithm

## Expectation Maximization Algorithm


**I**.EM算法是含有隐变量的概率模型极大似然估计或极大后验概率估计的迭代算法。含有隐变量的概率模型的数据表示为$\theta$.
$Y$ 是观测变量的数据,$Z$ 是隐变量, $\theta$是模型参数。EM算法通过迭代求解观测数据的对数似然函数${L}(\theta)=\log{P}(\mathrm{Y}|\theta)$
的极大化，实现极大似然估计。每次迭代包括两步：

1.E步：Expectation

求期望 $${L}(\theta) = \log{P}({Y}|{\theta}) => \sum_{Z} \log{P}(Y,Z | \theta) {P}(Z|Y, \theta^{(i)}) = Q(\theta, \theta^{(i)})$$
称为Q函数，这里$\theta^{(i)}$是参数的估计现值。


2.M步：Maximization
求极大化Q函数得到参数的新的估计值：

$$\theta^{(i)} = \arg \max_{\theta}Q(\theta, \theta^{(i)})$$

每次迭代中，EM算法通过极大化Q函数来增大对数似然函数$L(\theta)$



**II**.EM算法在每次迭代后均提高观测数据的似然函数值 $${P}({Y}|\theta^{(i+1)}) \geq {P}({Y}|\theta^{(i)})$$
在一般条件下EM算法是收敛的，但不能保证收敛到全局最优

**III**.EM算法应用极其广泛，主要应用于含有隐变量的概率模型学习。高斯混合模型的参数估计是EM算法的一个重要应用，隐马尔科夫模型的非监督学习也是EM算法的一个重要应用


**IV**.EM算法还可以解释为F函数的极大-极大算法。EM算法有许多变形，如GEM算法。GEM算法的特点是每次迭代增加F函数值（并不一定是极大化F函数），从而增加似然函数值。


**Eg**
>在统计学中，似然函数（likelihood function）是一个非常重要的内容，在非正式场合和概率（Probability）几乎是一对同义词。但是在统计学中似然和概率确实两个不同的概念。概率是指在特定环境下某件事情发生的可能性，也就是结果没有产生之前依据环境所对应的参数来预测某件事情发生的可能性。比如抛硬币，抛之前我们不知道最后是哪一面朝上，但是根据硬币的性质我们可以推测任何一面朝上的可能性均为50%，这个概率只有在抛硬币之前才是有意义的，抛完后结果便是确定的。而似然函数刚好相反，是在确定的结果下去推测产生这个结果的可能环境（参数）。还是抛硬币的例子，假设我们随机抛掷一枚硬币1000次，结果500次人头朝上，500次数字朝上，我们很容易跑断这是一枚标准硬币，两面朝上的galvanized均为50%。这个过程就是我们运用出现的结果来判断这个事情本身的性质（参数），也就是似然。


三硬币问题：
>假设有三枚硬币，分别记做A，B，C。这些硬币正面出现的概率分别是$\pi$, p和q。进行如下掷硬币实验：先投掷硬币A，正面选硬币B，反面选硬币C。然后掷选出的硬币，正面为1，反面为0；
> 独立地重复n次实验（n = 10）结果如下
> 1， 1， 0， 1， 0， 0， 1， 0 ，1， 1
> 假设只能观测到掷硬币的结果，无法观测到掷硬币的过程。问如何估计三硬币正面出现的概率

y为观测变量，表示实验结果为0 or 1；z为隐变量，表示未观测到的硬币A的结果
$${P}({Y}|\theta) = \sum_z{P}(y,z|\theta) = \sum_z{P}(y|z,\theta){P}(z|\theta) = \prod[\pi p^{y_i}(1-p)^{1-y_i} + (1 - \pi) q^{y_i}(1-q)^{1-y_i}]$$
$\theta$ = ($\pi$, p, q)为模型参数

E Step: 计算在模型参数$\theta^{i}$的前提下，观测数据y来自于硬币B的概率（硬币A为正面的概率）：

$$\mu^{i+1} = \frac{\pi ({p^{i}})^{y_i}(1-({p^{i}}))^{(1-y_i)}}{\pi ({p^{i}})^{y_i}(1-({p^{i}}))^{1-y_i} + (1 - \pi) ({q}^{i})^{y_i}(1-({q}^{i}))^{1-y_i}}$$


M step:
根据MOM，得到$\theta$的参数估计:
$\pi^{i+1} = \frac{1}{n} \sum_{j=1}^{n}\mu_j^{i+1}$
$p^{(i+1)} = \frac{\sum_{j=1}^{n}(\mu_j^{(i+1)}y_i)}{\sum_{j=1}^{n}\mu_j^{(i+1)}}$
$q^{(i+1)} = \frac{\sum_{j=1}^{n}(1 - \mu_j^{(i+1)})y_i}{\sum_{j=1}^{n}(1-\mu_j^{(i+1)})}$


In [10]:
# EM Algorithm to solve three coins problem
import numpy as np
import math

In [13]:
class EM:
    def __init__(self, prob_a, prob_b, prob_c):
        self.prob_a = prob_a
        self.prob_b = prob_b
        self.prob_c = prob_c

    def EStep(self, yi):
        p = self.prob_a * math.pow(self.prob_b, yi) * math.pow((1 - self.prob_b), 1-yi)
        q = (1 - self.prob_a) * math.pow(self.prob_c, yi)*math.pow((1 - self.prob_c), 1-yi)
        return p / (p + q)

    def MStep(self, y_array, mu_array):
        n = len(y_array)
        self.prob_a = np.sum(mu_array) / n
        self.prob_b = np.sum(y_array * mu_array) / np.sum(mu_array)
        self.prob_c = np.sum((np.array([1]*n) - mu_array) * y_array) / np.sum(np.array([1]*n) - mu_array)

    def fit(self, data, iter):
        y_array = np.array(data)
        for i in range(iter-1):
            mu = []
            for d in data:
                mu.append(self.EStep(d))
            mu_array = np.array(mu)
            self.MStep(y_array, mu_array)
        return [self.prob_a, self.prob_b, self.prob_c]


In [14]:
data = [1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
prob_list = EM(0.5,0.5,0.5).fit(data,2)
print(prob_list)


[0.5, 0.6, 0.6]


### Problems



> 1. 三硬币模型，假设观测数据不变，试选择不同的初值，例如：$\pi^{(0)} = 0.46,p^{(0)} = 0.55, q^{(0)} = 0.67$， 求参数模型为$\theta = (\pi, p, q)$的极大似然估计


In [16]:
data = [1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
prob_list = EM(0.46,0.55,0.67).fit(data,1000)
print(prob_list)



[0.4618628351139197, 0.5345950037850103, 0.6561346417857334]
